In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize 
from scipy.optimize import linprog
from gekko import GEKKO
import random

# Solving using the GEKKO API

In [9]:
# initializing and defining decision variables
def decision_variables(solver,n):
    """solver: as defined by GEKKO
        n : desired number of pipes
        output: decision variables with defined as per GEKKO solver"""
    random.seed(18)
    x0 = random.sample(range(1, n+1), n) # intitializing using random sample
    intitialized_decision_variables = []
    for y in range(n):
        y = solver.Var(value = x0[y], lb=1,ub=n,integer = True)
        intitialized_decision_variables.append(y)
    return intitialized_decision_variables

In [3]:
#single term computations as part of computing the objective function
# we get all the terms of interest from here
#y,p come from the table then var comes for the defining under GEKKO
def individual_pipe_expected_costs(dataframe,var):
    """ dataframe containing individual pipe water loss estimates
        and corresponding pipe leak probability estimate.
        var : instantiated variables according to the number of decision 
             variables.
       Output: list with individual pipe costs"""
    y = list(dataframe.Estimated_water_loss)
    p = list(dataframe.Pipe_leak_probability)
    return [var[t]*y[t]*p[t] for t in range(len(var))]    

In [7]:
def optimal_maintenance_sequence(dataframe):
    """Assuming daily updates, this optimization script can be run preferably in the morning
        to provide an optimal sequence of how to go about fixing pipes in the distribution
        system in order to minimize water losses.
        Inputs a dataframe with details on estimated water losses and probility of leaks
        for individual pipes in the distribution.
    """
    sequence = []
    n = len(dataframe)
    M =n*100
    episilon = 0.03    
    m = GEKKO()
    m.options.SOLVER=1
    dcv = decision_variables(m,n) # declaring dv's n will come from the table passed on to us
    objective_function_terms = individual_pipe_expected_costs(dataframe,dcv)
    # Constraints 
    m.Equation(sum(dcv) ==  (n*(n+1))/2)
    for i in range(n):
            for j in range(n):                
                if i<j:
                    delta = 1
                    m.Equation(abs(dcv[i] - dcv[j]) <= (-1* episilon) + (M * delta))
                    m.Equation(abs(dcv[i] - dcv[j]) >= episilon - (1-delta)* M)
    m.Obj(sum(objective_function_terms))
    m.solve(disp = False)
    for d in range(len(dcv)):
        sequence.append(dcv[d].value)
    return(sequence)

### Source for technique to handle the constraint $x_i \neq x_j$. 

https://math.stackexchange.com/questions/37075/how-can-not-equals-be-expressed-as-an-inequality-for-a-linear-programming-model/1517850

In [10]:
# Test_data

columns = ['Pipe_id','Estimated_water_loss','Pipe_leak_probability']
rows = [['P1',1.5,0.35],['P2',1.25,0.4],['P3',1.1,0.9],['P4',1.4,0.6],['P5',1.45,0.5]]
df1 = pd.DataFrame(rows, columns = columns)

In [11]:
optimal_maintenance_sequence(df1)

[[4.0], [5.0], [2.0], [1.0], [3.0]]